In [1]:
import sys
sys.path.append('../TTI/')
%load_ext autoreload
%autoreload 2

# Taksonomia, identyfikacja tekstu

Dany jest fragment hierarchii klasyfkacji tematycznej z Wikipedii (https://en.wikipedia.org/wiki/Category:Main_topic_classifications) w postaci pliku CSV.
Klasyfkacja jest grafem spójnym, gdzie węzły są tematami, a krawędzie reprezentują uszczegółowienie tematu.

Celem projektu jest zapropnowanie i przetestowanie mechanizmu automatycznej klasyfikacji tekstu Wejściem jest plik tekstowy w języku angielskim. Wyjściem jest zbiór węzłów w/w klasyfikacji tematycznej.


## Dane wejściowe

Dane wejściowe do zadania do graf spójny o 225765 węzłach, kady węzeł reprezentuje jedną kategorię. Graf nie jest uporządkowanym drzewem, może również zawierać pętle.

In [2]:
from TTI.CategoriesGraph import CategoriesGraph

categories = CategoriesGraph()

Reading topics graph


In [3]:
print("Ilość krawędzi", categories._edge_list.shape)
print("Ilość węzłów", categories._graph.number_of_nodes())

Ilość krawędzi (339250, 2)
Ilość węzłów 225765


## Zbiór treningowy

Zbiór treningowy został przygotowany z wykorzystaniem notebooka `01-tti-training-set-generate.ipynb`. Tam jest też więcej informacji o procesie generacji.

In [6]:
from TTI.config import DATABASE_PATH
import sqlite3
import pandas as pd
import json
import numpy as np
from tensorflow.keras.utils import to_categorical

table_name = "training_set_25"
connection = sqlite3.connect(DATABASE_PATH)


In [7]:
dataset = pd.read_sql('select * from {}'.format(table_name), connection)

In [8]:
dataset["Representation"] = dataset["Representation"].apply(lambda i : json.loads(i))
dataset["Category"] = dataset["Category"].apply(lambda i : i[9:])
dataset["Words"] = dataset["Words"].apply(lambda i : json.loads(i))

In [9]:
print("Dataset size:", dataset.shape)
print("Numeric represntation vector size:", len(dataset.iloc[12]["Representation"]))
print("Number of nodes in the graph:", len(dataset.iloc[12]["Words"]))

Dataset size: (225765, 3)
Numeric represntation vector size: 300
Number of nodes in the graph: 25


In [10]:
dataset

,Category,Words,Representation
0,Main_topic_classifications,"[academic, culture, human, entertainment, heal...","[-0.3755445182323456, 0.010519789531826973, -0..."
1,Main topic articles,"[academic, culture, human, entertainment, heal...","[-0.40671899914741516, 0.013835961930453777, -..."
2,Academic disciplines,"[academic, art, academics, euthenics, studies,...","[-0.09239675104618073, -0.46590009331703186, -..."
3,Subfields by academic discipline,"[subfield, academic, areas, evolutionary, fiel...","[0.085173599421978, 0.010392077267169952, -0.3..."
4,Scholars by subfield,"[subfield, academic, architects, studies, clas...","[-0.15292514860630035, -0.5975006222724915, -0..."
...,...,...,...
225760,World Wide Web stubs,"[internet, wide, system, technology, bioinform...","[0.216136172413826, -0.024581177160143852, -0...."
225761,Internet publication stubs,"[service, wide, entertainment, online, news, s...","[0.2748589515686035, 0.2310565859079361, -0.34..."
225762,Website stubs,"[websites, service, wide, entertainment, onlin...","[0.1632257103919983, 0.16291794180870056, -0.2..."
225763,Wikimedia Foundation stubs,"[websites, service, wide, entertainment, onlin...","[0.19932252168655396, 0.19686073064804077, -0...."


## Wyszukiwanie najbardziej podobnych wektorów

Do klasyfikacji posłuże się obliczaniem odległości geometrycznej pomiędzy wektorami reprezentacji doc2vec. Wektory o najmniejszej odległości zostaną zakwalifikowane jako najbardziej podobne.

In [32]:
dataset.loc[dataset['Category'] == "Machine learning algorithms"]


,Category,Words,Representation
2692,Machine learning algorithms,"[checksum, algorithmic, trading, compression, ...","[0.302137166261673, 0.3030090630054474, -1.029..."


In [34]:
from scipy import spatial

name = dataset["Category"][2692]
vector = dataset["Representation"][2692]

print("Name of category", name)

Name of category Machine learning algorithms


Teraz należe znaleźć najbardziej podobne kategorie. 

In [35]:
import tqdm

def find_simmilar(vector, count, df):
    """ Finds 'count' best matching categories with vectors simmilar to 'vector'"""
    categories = []
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
        vec = row["Representation"]
        name = row["Category"]
        categories.append((name, spatial.distance.cosine(vector, vec)))
    sorted_categories = sorted(categories, key=lambda i: i[1])
    return sorted_categories[0:count]


In [36]:
best_matching = find_simmilar(vector, 10, dataset)

100%|██████████| 225765/225765 [00:38<00:00, 5924.71it/s]


In [37]:
best_matching

[('Machine learning algorithms', 0.0),
 ('Heuristic algorithms', 0.03048611901635534),
 ('Cryptographic algorithms', 0.033841232971325574),
 ('Computer arithmetic algorithms', 0.034831473629752474),
 ('Data mining algorithms', 0.03487279219354722),
 ('Compression algorithms', 0.035516547716642144),
 ('Digit-by-digit algorithms', 0.03591962687260464),
 ('Algorithms', 0.03608325172873006),
 ('Bioinformatics algorithms', 0.03669657755224942),
 ('Approximation algorithms', 0.03737963968331992)]

In [39]:
best_matching[1]

('Heuristic algorithms', 0.03048611901635534)

# -----------------------

In [35]:
testcases_count, _ = dataset.shape

In [52]:
X = X[:10000]
Y = Y[:10000]

In [93]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=1000, random_state=0).fit(X)

In [110]:
X[2345].shape

(300,)

In [105]:
print(dataset["Category"][2345])
print(kmeans.predict(X[2345][:]))

Category:Computational fields of study


ValueError: Expected 2D array, got 1D array instead:
array=[-1.05711957e-03 -1.12174821e-04 -6.70599227e-04  2.00716982e-04
 -1.12532716e-05 -1.19818910e-03 -1.00701256e-03  1.51766115e-03
  1.38851278e-03  1.20256539e-03 -1.02999003e-03  1.59012235e-03
 -5.12395869e-04  1.04147161e-03 -1.88910679e-04 -1.18044345e-03
  3.29556147e-04  3.28550494e-04  7.17254938e-04  1.46737450e-03
  8.61251028e-04 -5.69134543e-04  1.21984247e-03 -7.90496066e-04
 -2.82717112e-04 -1.40814681e-03  3.09491617e-04 -4.13069793e-04
 -1.36248779e-03 -3.97055119e-05 -2.56790023e-04  1.99193833e-04
  4.57675749e-04  1.64242432e-04 -1.70118772e-04  8.39911751e-04
  1.49876974e-03 -1.35887484e-03 -1.46193802e-03 -1.29698543e-03
  1.13061490e-03  4.60446638e-04 -8.44697934e-04 -4.18586103e-04
  3.92560614e-04 -1.60691852e-03  1.19609770e-03 -1.41348923e-03
  1.08640420e-03 -8.45222457e-05 -1.46952469e-03  1.60608278e-03
 -4.27623512e-04  1.15115463e-03  2.16798435e-05  1.13619689e-03
  3.14626013e-05  1.47080002e-0

In [91]:
clasterCategoryMap = {}
for index, value in enumerate(kmeans.labels_):
    if value in clasterCategoryMap:
        clasterCategoryMap[value].add(dataset["Category"][index])
    else:
        clasterCategoryMap[value] = set([dataset["Category"][index]])

In [92]:
clasterCategoryMap[11]

{'Category:(ISC)²',
 'Category:Abstract data types',
 'Category:Action theorists',
 'Category:Administrative terminology',
 'Category:Aerospace engineering award winners',
 'Category:Aerospace engineers',
 'Category:Agent-based software',
 'Category:Agroecology',
 'Category:Aircraft designers',
 'Category:Algorithmic information theory',
 'Category:Alternative medicine publications',
 'Category:American civil engineering contractors',
 'Category:Animal fats',
 'Category:Anthropological categories of peoples',
 'Category:Archaeologists',
 'Category:Archaeologists appearing on Time Team',
 'Category:Archaeologists by century',
 'Category:Art history',
 'Category:Artificial intelligence researchers',
 'Category:Asphalt lakes',
 'Category:Atmospheric sciences',
 'Category:Belgian historians of religion',
 'Category:Belief revision',
 'Category:Biblical authorship debates',
 'Category:Binary arithmetic',
 'Category:Biogeographic realms',
 'Category:Biological anthropology',
 'Category:Biolo

In [80]:
print(dataset["Words"][10])
print(dataset["Category"][10])

['archaeologist', 'anthropologists', 'fictional', 'subfield', 'or', 'armenian', 'archaeology', 'appearing', 'sinhalese', 'field', 'antiquarians', 'nationality', 'women', 'indigenous', 'ethnic', 'arab', 'by', 'burgher', 'people', 'crimean', 'igbo', 'national', 'catalan', 'tamil', 'of', 'flemish', 'occupations', 'archaeologists', 'australian', 'american', 'history', 'team', 'scientists', 'ethnicity', 'time', 'geoarchaeologists', 'origin', 'on', 'in', 'iranian', 'academics', 'basque', 'tatar', 'black', 'social', 'lists', 'stubs', 'research', 'british', 'century']
Category:Archaeologists by ethnicity


In [53]:
from sklearn.preprocessing import LabelEncoder

train_y = dataset["Category"][:10000]

label_encoder = LabelEncoder()
label_encoder.fit(train_y)
Y = to_categorical((label_encoder.transform(train_y)))

In [59]:
X = np.array([np.array(xi) for xi in dataset["Representation"][:10000]])
numeric_vector_size = len(X[0])

In [60]:
Y.shape

(10000, 10000)

In [67]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(numeric_vector_size,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10000, activation='softmax'))

print(model.summary())


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 256)               77056     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 10000)           

In [68]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
history = model.fit(X, Y, epochs=200, batch_size=256)

Epoch 1/200
40/40 [==============================] - 2s 38ms/step - loss: 9.2134 - accuracy: 0.0000e+00
Epoch 2/200
40/40 [==============================] - 1s 37ms/step - loss: 9.2111 - accuracy: 0.0000e+00
Epoch 3/200
40/40 [==============================] - 1s 37ms/step - loss: 9.2112 - accuracy: 0.0000e+00
Epoch 4/200
40/40 [==============================] - 1s 37ms/step - loss: 9.2113 - accuracy: 0.0000e+00
Epoch 5/200
40/40 [==============================] - 2s 42ms/step - loss: 9.2112 - accuracy: 0.0000e+00
Epoch 6/200
40/40 [==============================] - 2s 44ms/step - loss: 9.2113 - accuracy: 1.2533e-04
Epoch 7/200
40/40 [==============================] - 2s 43ms/step - loss: 9.2114 - accuracy: 1.9259e-04
Epoch 8/200
40/40 [==============================] - 2s 43ms/step - loss: 9.2112 - accuracy: 3.2688e-05
Epoch 9/200
40/40 [==============================] - 2s 41ms/step - loss: 9.2113 - accuracy: 2.0574e-05
Epoch 10/200
40/40 [==============================] - 2s 42ms/st

KeyboardInterrupt: 

In [15]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.decomposition import PCA

# pca = PCA(n_components=200)
# pca.fit(X)
# print(sum(pca.explained_variance_ratio_))


In [16]:
# model = KNeighborsClassifier(n_neighbors=3)

# model.fit(X, Y)

## Zapisywanie modelu

In [ ]:
# import cPickle

# # save the classifier
# with open('models/knn.pkl', 'wb') as fid:
#     cPickle.dump(model, fid)    
